In [5]:
import tweepy
from tqdm.auto import tqdm
from IPython.display import display, HTML
import re
import math
import requests
import os


def get_and_sortdesc(sort_criteria, get_f, id, total=None, **kwargs):
    sorted_ls = []
    for obj in tqdm(
        tweepy.Paginator(get_f, id, user_auth=True, **kwargs).flatten(), total=total
    ):
        insert_ind = 0
        for ind, sorted_obj in enumerate(sorted_ls):
            if sort_criteria(obj) >= sort_criteria(sorted_obj):
                insert_ind = ind
                break
        sorted_ls.insert(insert_ind, obj)
    return sorted_ls

### tweepy client

tweepy v4
https://docs.tweepy.org/en/stable/client.html

In [3]:
API_KEY = os.getenv("TWITTER_BOT_KEY")
API_SECRET_KEY = os.getenv("TWITTER_BOT_SECRET_KEY")

ACCESS_TOKEN = os.getenv("reebott_ACCESS_TOKEN")
ACCESS_TOKEN_SECRET = os.getenv("reebott_ACCESS_TOKEN_SECRET")


client = tweepy.Client(
    consumer_key=API_KEY,
    consumer_secret=API_SECRET_KEY,
    access_token=ACCESS_TOKEN,
    access_token_secret=ACCESS_TOKEN_SECRET,
    wait_on_rate_limit=True,
)
me = client.get_me(user_fields=["profile_image_url"]).data
display(HTML(f"""<img src={me.profile_image_url}>{me}"""))

In [4]:
target = client.get_user(
    username="umbrecore",
    user_fields=["public_metrics", "description", "profile_image_url"],
    user_auth=True,
).data
display(
    HTML(
        f"""<!--img src={target.profile_image_url}-->
{target.description}<br>
{target.public_metrics}"""
    )
)

**Parameters**

`max_results`: results per page
- `.get_users_followers()`: 1 <= `max_results` <= 1000
- `.get_users_tweets()`: 1 <= `max_results` <= 100

`limit`: max number of pages

### Followers insight using `.get_users_followers()`

#### Get

In [6]:
mr = 1000
sorted_flwrs = get_and_sortdesc(
    lambda user: user.public_metrics["followers_count"],
    client.get_users_followers,
    target.id,
    total=target.public_metrics["followers_count"],
    max_results=mr,
    limit=target.public_metrics["followers_count"] // mr + 1,
    user_fields=["public_metrics", "profile_image_url"],
)

  0%|          | 0/2249 [00:00<?, ?it/s]

In [18]:
def user_filterer(user):
    if user.public_metrics["following_count"] < 100:
        return True
    if "___" in user.name:
        return True


for user in filter(user_filterer, sorted_flwrs):
    print(f"@{user.username}\t\t{user.public_metrics['followers_count']}\t{user.name}")

@Sky_Cloud69		3101	☁️
@dacoool		1852	Dac
@auralabsart		548	Kuro (Commissions: Open)
@SylveonUmb		307	Umbreon&Sylveon Pokeganda
@pokegirl_s		191	Hental
@TheTibbtTile		62	twibbter
@wheeled_tank		54	Underpable
@AquamarineRose_		48	AquamarineRose (Formerly CataruMoore)
@Masuda_heart		47	Gravy
@nyuwech		35	Nyuwech 🔜🇬🇧
@HisuiBunny		34	ProxyRabbit
@HydratheLucario		32	Hydra
@Spooky4221		30	StarFlight
@galaxyflareon1		28	galaxy eeveelutions
@FireballKitsuAD		23	✧Fireball✧Kitsune✧AD
@WinterfoxArt		22	Ren
@lewdingPanda		19	A Rather Lewd Panda
@Purple_Sylveon		19	Purple_Sylveon
@RedWotter		18	Rotted
@AstralSparx		17	Sparx
@teachii6		17	tea
@Roman_god24		17	littleRoman Studios
@XReassure		16	Reassure
@VariElFurro		16	Vari El Furro
@liberators_fate		14	Lance, o Riolu
@TailsCuteVN		14	Toul The Fox (Opening)
@YANTAB3		14	YAN TAB
@ExKeldeo		13	LífNeon
@Lil_Kira_flower		13	Lil' Kira
@BlindingWulf		13	Blinding Wulf
@Zveltal		12	ZveL
@2021Sobble		11	Sobble gamer 2021
@pokerpokedd		10	Poker / looking for 

#### Output to file

In [10]:
html_ = f"<h2>{target.name}'s top 100 followers by popularity</h2>"
for i, user in enumerate(sorted_flwrs, 1):
    flwrs_cnt = user.public_metrics["followers_count"]
    if i > 100:
        break
    bold = flwrs_cnt > 100000
    html_ += f"""
        <div {'style="color:red;"' if bold else ''}>
            <b>{i}.</b><br>
            <a href='https://twitter.com/{user.username}'>
                <img src={user.profile_image_url}><br>
                {user.name} (@{user.username})
            </a>
            <div>
                {flwrs_cnt} followers<br>
                {user.public_metrics['following_count']} followings
            </div>
        </div>
"""
# display(HTML(html_))
with open(f"../../Output/{target.username}_followers.html", "w", encoding="utf-8") as f:
    f.write(html_)

### Analiyze

In [ ]:
sorted_flwgs = get_and_sortdesc(
    lambda user: user.public_metrics["followers_count"],
    client.get_users_following,
    target.id,
    max_results=1000,
    user_fields=["public_metrics", "profile_image_url"],
)

##### Mutuals

In [ ]:
sorted_mutuals = [u for u in sorted_flwrs if u in sorted_flwgs]
print(sorted_mutuals.__len__())

##### Haters

In [ ]:
sorted_likers = get_and_sortdesc(
    lambda user: user.public_metrics['followers_count'],
    client.get_liking_users,
    1541858974756065280,
    user_fields=['public_metrics', 'profile_image_url']
)

In [ ]:
sorted_likers[:10]

In [ ]:
haters = [u for u in sorted_flwrs if u not in sorted_likers]
haters[:50]

### Tweets insight using `.get_users_tweets()`

#### Get

In [ ]:
mr = 100
lim = target['public_metrics']['tweet_count'] // mr + 1
# lim = 1
tweet_list = [tweet for tweet in tqdm(tweepy.Paginator(
    client.get_users_tweets,
    # client.get_liked_tweets,
    target['id'], tweet_fields=['in_reply_to_user_id'], max_results=mr, limit=lim, user_auth=True
).flatten())]

#### Get one

In [ ]:
tweet_list[0]

In [ ]:
res = requests.get('https://cdn.syndication.twimg.com/tweet?id=1581735723992256513').json()
res

In [ ]:
display(HTML(f'<video src="{res["video"]["variants"][0]["src"]}">'))

#### Filter

In [ ]:
def filterer(tweet):
    txt = tweet.text.lower()
    # if "http" in txt:
    #     return False
    if "rt @" in txt:
        return False
    # if "ai " not in txt:
    #     return False
    # if tweet.in_reply_to_user_id is not None:
    #     return True
    return True


for tweet in filter(filterer, tweet_list):
    print(tweet.id, tweet.text)


In [ ]:
client.delete_tweet(1520825930129633280)

#### Analyze

In [ ]:
user_dict = {}
for tweet in tqdm(tweet_list):
    text = tweet.text
    text = text.split()
    text = ' '.join([word for word in text if not word.startswith('@')])
    # remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    words = text.split()
    for word in words:
        if word in user_dict:
            user_dict[word] += 1
        else:
            user_dict[word] = 1

In [ ]:
# sort dictionary by value
sorted_dictionary = sorted(user_dict.items(), key=lambda kv: kv[1], reverse=True)
sorted_dictionary